In [117]:
from pathlib import Path
from anytree import Node, RenderTree

# read file
file = open(Path("input.txt"), 'r')
commands = file.read().split("\n")

# set the cursor to a new node
def set_cursor(to, current):
    for n in current.children:
        if n.name == to:
            return n

# create directory and file tree
root = Node("/", size=0, file=False)
cursor = root
for command in commands:
    if "$ cd " in command:
        if "/" in command:
            cursor = root
        elif ".." in command:
            cursor = cursor.parent
        else:
            cursor = set_cursor(command.split("cd ")[1], cursor)
    elif "dir " in command:
        temp = Node(command.split("dir ")[1], parent=cursor, size=0, file=False)
    elif "$ ls" in command:
        pass
    else:
        size, name = command.split(" ")
        f = Node(name, parent=cursor, size=size, file=True)

# print file/directory tree
# for pre, fill, node in RenderTree(root):
#    print("%s%s" % (pre, node.name))

# calc directory sizes
def calc_size(root_node):
    s = 0
    for n in root_node.children:
        if n.file:
            s += int(n.size)
        elif not n.file:
            n.size = calc_size(n)
            s += int(n.size)
    return s

# get directories with size x
def get_directories_not_larger_than(root_node, x=100000):
    temp = []
    for n in root_node.children:
        if (int(n.size) <= x) & (n.file == False):
            temp.append(n)
        if len(n.children) > 0:
            temp.append(get_directories_not_larger_than(n, x))
    return temp

def get_directories_larger_than(root_node, x=100000):
    temp = []
    for n in root_node.children:
        if (int(n.size) > x) & (n.file == False):
            temp.append(n)
        if len(n.children) > 0:
            temp.append(get_directories_larger_than(n, x))
    return temp

# calculate each directories size
root.size = calc_size(root)
# find directories which are smaller than x
dir_list = get_directories_not_larger_than(root, x = 100000)
# flatten the list of directories returned
flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
dir_list = flatten(dir_list)

# solve
# calculate total size of directories smaller than x
total_size = 0
for d in dir_list:
    total_size += d.size
print("Solution part 1:\nThe total size of directories with a size smaller than 100000 is:", total_size)

print(root.size)
unused_space = 70000000 - root.size
update_size = 30000000
space_to_free = update_size - unused_space
print("------------------\nSolution part 2:\nUnused space before deletion:", unused_space)
print("needs free space:", space_to_free)
dir_list = flatten(get_directories_larger_than(root, x = space_to_free))
# find minimum
min_size_deletion = root.size
for d in dir_list:
    if (int(d.size) < min_size_deletion) & (int(d.size) > space_to_free):
        min_size_deletion = int(d.size)
print("Minimum deletion size is:", min_size_deletion)

Solution part 1:
The total size of directories with a size smaller than 100000 is: 1581595
41518953
------------------
Solution part 2:
Unused space before deletion: 28481047
needs free space: 1518953
Minimum deletion size is: 1544176
